![price-and-cut-algo](price-and-cut-algo.png)

In [12]:
import pulp

# --- 1. FAKE DATA (To make the example runnable) ---
# In a real problem, this 'known_schedules' list would be
# populated by the Subproblem.

# We represent each "column" (schedule j) as a Python dictionary
schedule_1 = {
    "id": "Sched_1_Mon",
    "B_j": 240,  # Total minutes (profit) of surgeries
    "day": "Mon",
    "surgeries": [101, 102],  # List of surgery IDs
    # Eq (5) data: {surgeon: total_minutes}
    "surgeon_work": {"Dr_A": 120, "Dr_B": 120},
    # Eq (6) data: {(surgeon, day, time_slot): 1 if busy}
    "surgeon_busy_times": {
        ("Dr_A", "Mon", 9): 1, ("Dr_A", "Mon", 10): 1,  # Dr_A busy 9-11
        ("Dr_B", "Mon", 11): 1, ("Dr_B", "Mon", 12): 1   # Dr_B busy 11-1
    }
}

schedule_2 = {
    "id": "Sched_2_Mon",
    "B_j": 300,
    "day": "Mon",
    "surgeries": [103],  # A mandatory surgery
    "surgeon_work": {"Dr_A": 300},
    "surgeon_busy_times": {
        ("Dr_A", "Mon", 11): 1, ("Dr_A", "Mon", 12): 1, ("Dr_A", "Mon", 13): 1,
        ("Dr_A", "Mon", 14): 1, ("Dr_A", "Mon", 15): 1   # Dr_A busy 11-4
    }
}

# The list of all columns the Master Problem currently knows about
known_schedules = [schedule_1, schedule_2]

# --- FAKE Problem Parameters ---
MANDATORY_SURGERIES = [103, 101, 102]             # Set I_1
OPTIONAL_SURGERIES = []         # Set I_2
ALL_SURGEONS = ["Dr_A", "Dr_B"]         # Set L
ALL_DAYS = ["Mon", "Tue"]               # Set D
ALL_TIMES = [9, 10, 11, 12, 13, 14, 15, 16] # Set T_d

# Resource Limits (RHS of constraints)
K_d = {"Mon": 5, "Tue": 5}  # Max 5 ORs per day (set of ORs for day d)
A_ld = {  # Max minutes for surgeon 'l' on day 'd'
    ("Dr_A", "Mon"): 480, ("Dr_B", "Mon"): 480,
    ("Dr_A", "Tue"): 480, ("Dr_B", "Tue"): 480,
}


def build_master_lp(known_schedules, mandatory_surgeries, optional_surgeries,
                    all_surgeons, all_days, all_times, K_d, A_ld):
    """
    Builds the Master Problem LP model from the paper.
    """

    # --- 1. Create the Problem ---
    prob = pulp.LpProblem("Master_Problem_LP", pulp.LpMaximize)

    # --- 2. Create Variables (x_j) ---
    # x_j = 1 if schedule j is used. Solved as LP (Continuous).
    # This is variable x_j from [cite: 166]
    sched_vars = pulp.LpVariable.dicts(
        "Schedule",
        [s["id"] for s in known_schedules],
        lowBound=0,
        cat='Continuous'
    )

    # --- 3. Objective Function (Eq. 1) ---
    # Maximize total scheduled surgery time: Max sum(B_j * x_j) 
    prob += pulp.lpSum(
        s["B_j"] * sched_vars[s["id"]] for s in known_schedules
    ), "Total_Surgery_Time"

    # --- 4. Constraints (This is where the dual prices come from) ---

    # === Constraint (2) & (3) -> dual price is pi_i ===
    # (2) Mandatory: sum(x_j for j containing i) = 1 [cite: 189, 190]
    for i in mandatory_surgeries:
        prob += pulp.lpSum(
            sched_vars[s["id"]] for s in known_schedules if i in s["surgeries"]
        ) == 1, f"Pi_i_Mandatory_{i}" # <-- Name generates the pi

    # (3) Optional: sum(x_j for j containing i) <= 1 [cite: 189, 190]
    for i in optional_surgeries:
        prob += pulp.lpSum(
            sched_vars[s["id"]] for s in known_schedules if i in s["surgeries"]
        ) <= 1, f"Pi_i_Optional_{i}" # <-- Name generates the pi

    # === Constraint (4) -> dual price is pi_d ===
    # Number of schedules on day d <= Number of ORs on day d [cite: 189, 192]
    for d in all_days:
        prob += pulp.lpSum(
            sched_vars[s["id"]] for s in known_schedules if s["day"] == d
        ) <= K_d[d], f"Pi_d_OR_Limit_{d}" # <-- Name generates the pi

    # === Constraint (5) -> dual price is pi_ld ===
    # Total surgeon work on day d <= Max hours for surgeon on day d [cite: 189, 193]
    for l in all_surgeons:
        for d in all_days:
            prob += pulp.lpSum(
                s["surgeon_work"].get(l, 0) * sched_vars[s["id"]]
                for s in known_schedules if s["day"] == d
            ) <= A_ld[(l, d)], f"Pi_ld_Surgeon_Hours_{l}_{d}" # <-- Name

    # === Constraint (6) -> dual price is pi_ldt ===
    # Prevents surgeon overlap [cite: 189, 193]
    # sum(x_j if surgeon l is busy at time t on day d in schedule j) <= 1
    for l in all_surgeons:
        for d in all_days:
            for t in all_times:
                prob += pulp.lpSum(
                    # s["surgeon_busy_times"].get((l, d, t), 0) checks if
                    # schedule 's' uses surgeon 'l' at day 'd', time 't'
                    s["surgeon_busy_times"].get((l, d, t), 0) * sched_vars[s["id"]]
                    for s in known_schedules if s["day"] == d
                ) <= 1, f"Pi_ldt_Surgeon_Overlap_{l}_{d}_{t}" # <-- Name

    return prob

# --- Main function to run the example ---
if __name__ == "__main__":

    # --- Build the LP ---
    # master_lp = build_master_lp(known_schedules, MANDATORY_SURGERIES, OPTIONAL_SURGERIES,
    #                             ALL_SURGEONS, ALL_DAYS, ALL_TIMES, K_d, A_ld)

    # # --- Solve the LP ---
    # print("Solving the Master LP to get dual prices...")
    # master_lp.solve()
    # print(f"Status: {pulp.LpStatus[master_lp.status]}\n")

    # # --- Extract and Print Dual Prices ---
    # print("--- DUAL PRICES (pi) ---")
    # print("These are the values sent to the Subproblem.\n")

    # # We store the dual prices (pi) in a dictionary
    # dual_prices = {}
    # for name, c in master_lp.constraints.items():
    #     # c.pi IS the dual price for that constraint
    #     dual_prices[name] = c.pi

    # print("=== Prices for pi_i (Surgery Constraints) ===")
    # for i in MANDATORY_SURGERIES:
    #     print(f"  Pi_i_Mandatory_{i}: {dual_prices[f'Pi_i_Mandatory_{i}']}")
    # for i in OPTIONAL_SURGERIES:
    #     print(f"  Pi_i_Optional_{i}: {dual_prices[f'Pi_i_Optional_{i}']}")

    # print("\n=== Prices for pi_d (OR Day Limits) ===")
    # for d in ALL_DAYS:
    #     print(f"  Pi_d_OR_Limit_{d}: {dual_prices[f'Pi_d_OR_Limit_{d}']}")

    # print("\n=== Prices for pi_ld (Surgeon Hour Limits) ===")
    # for l in ALL_SURGEONS:
    #     for d in ALL_DAYS:
    #         print(f"  Pi_ld_Surgeon_Hours_{l}_{d}: {dual_prices[f'Pi_ld_Surgeon_Hours_{l}_{d}']}")

    # print("\n=== Example Prices for pi_ldt (Surgeon Overlap) ===")
    # print(f"  Pi_ldt...Dr_A_Mon_9: {dual_prices['Pi_ldt_Surgeon_Overlap_Dr_A_Mon_9']}")
    # print(f"  Pi_ldt...Dr_A_Mon_11: {dual_prices['Pi_ldt_Surgeon_Overlap_Dr_A_Mon_11']}")
    # print(f"  Pi_ldt...Dr_B_Mon_11: {dual_prices['Pi_ldt_Surgeon_Overlap_Dr_B_Mon_11']}")

    
    # 1. Build the problem one last time, with ALL 500 schedules
    final_prob = build_master_lp(known_schedules, MANDATORY_SURGERIES, OPTIONAL_SURGERIES,
                                ALL_SURGEONS, ALL_DAYS, ALL_TIMES, K_d, A_ld)

    # 2. Change all variables to be Integer
    for v in final_prob.variables():
        v.cat = 'Integer'
        # You'd also set an upper bound
        v.upBound = 1 # We can only use a specific daily schedule once

    # 3. Solve this as an Integer Problem
    print("Solving the FINAL Integer Problem for the real answer...")
    final_prob.solve()

    # --- CHECK THE STATUS ---
    print(f"Status: {pulp.LpStatus[final_prob.status]}")

    # 4. THIS IS YOUR REAL ANSWER
    if final_prob.status == pulp.LpStatusOptimal:
        print("\n--- FINAL OR ROOM ALLOCATION ---")
        for v in final_prob.variables():
            if v.value() > 0.9: # i.e., if the variable = 1.0
                print(f"USE SCHEDULE: {v.name}")
    else:
        print("\nNo valid schedule allocation found.")
        print("The problem is likely infeasible.")
   

Solving the FINAL Integer Problem for the real answer...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/matthewkeller/4A/bme-411/411-final-project/or-env/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/_m/zyrbphls3w34fhsvk8rqt6880000gn/T/0f52af5ef9fc494796c31195e48c8eb7-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/_m/zyrbphls3w34fhsvk8rqt6880000gn/T/0f52af5ef9fc494796c31195e48c8eb7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 46 COLUMNS
At line 70 RHS
At line 112 BOUNDS
At line 115 ENDATA
Problem MODEL has 41 rows, 2 columns and 17 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 540 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node cha

In [ ]:
# 1. Start with a few initial, simple schedules (from your heuristic)
known_schedules = get_initial_schedules()
new_schedules_found = True

while new_schedules_found:
    
    # --- Step 1 (Master Problem) ---
    # Build and solve the LP with all schedules found so far
    master_lp = build_master_lp(known_schedules) 
    master_lp.solve()

    # Get the dual prices for all constraints
    dual_prices = extract_dual_prices(master_lp) 
    
    # --- Step 2 (Subproblem) ---
    new_schedules_found = False
    
    # Solve one subproblem for each day
    for day in ALL_DAYS:
        # Pass the dual prices to the Subproblem
        subproblem_cp = build_subproblem_cp(day, dual_prices)
        subproblem_cp.solve()
        
        # The objective value *is* the reduced cost
        max_reduced_cost = subproblem_cp.ObjectiveValue()
        
        # --- Step 3 (The Check) ---
        # If the "profit" (reduced cost) is positive,
        # we found a good new schedule!
        if max_reduced_cost > 0.00001:
            new_schedules_found = True
            
            # Get the new schedule from the CP solver
            new_schedule_j = extract_schedule_from_cp(subproblem_cp)
            
            # Add the new schedule to our master list
            known_schedules.append(new_schedule_j)

# Loop is over! We have converged.
print("Column Generation Complete.")